Click the play button next to the code snippet below to load all tools we'll need. Scroll down, and you'll be asked whether you want to use the Sandbox environment (for testing) or the Production environment (using 'live' data from Alma).

You'll then be asked for your list of MMSIDs - this should be a text file (.txt), with an MMSID on each line. If you copy a list of MMSIDs from excel, paste it into notepad and save it somewhere, this will be exactly what you need.

In [ ]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, FileLink
import ipywidgets as widgets

def clicked_sand(arg):
    global apiKey
    apiKey = os.environ['API_KEY_SAND']
    global apiType
    apiType = 'Sandbox'
    with out:
        print('\rUsing Sandbox data   ', end='')

def clicked_prod(arg):
    global apiKey
    apiKey = os.environ['API_KEY_PROD']
    global apiType
    apiType = 'Production'
    with out:
        print('\rUsing Production data', end='')

print('Would you like to use the Sandbox or Production environment?')
button_sand = widgets.Button(description = 'Sandbox', tooltip='Sandbox')   
button_sand.on_click(clicked_sand)

button_prod = widgets.Button(description = 'Production', tooltip='Production')   
button_prod.on_click(clicked_prod)

out = widgets.Output()
buttons = widgets.HBox([button_sand, button_prod])
outdisplay = widgets.VBox([out])
uploader = widgets.FileUpload(description ='Upload file', accept='.txt', multiple=False)
display(buttons, outdisplay)
# print('Upload your MMSID txt file here - this should be a text file (.txt), with an MMSID on each line')
# display(uploader)


widgetInput = widgets.Textarea(value='Hello World textarea', placeholder='Type something', description='described', disabled=False)
display(widgetInput)

Run the next code snippet, and then paste your list of MMSIDs into the box that appears. They should automatically format from a vertical list to a single line.

In [ ]:
MMSIDs = input('paste your list of MMSIDs here')
IDlist = MMSIDs.split()

Finally, run the code snippet below. You should get a running total of the completed files, and at the end you'll see a dataframe of the output, and a button to copy it all (for pasting into excel or similar)

In [ ]:
start = time.time()
bCount = len(IDlist)
hCount = 1

for bibID in IDlist:
    url = f'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/bibs/{bibID}?view=full&expand=None&' \
          f'apikey={apiKey}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'xml')
    vols = soup.find(tag='300').find(code='a')
    fname = f'output_{str(start)[-4:]}_{apiType}.txt'
    with open(fname, 'a', encoding='utf-8') as f:
        f.write(f'{bibID}\t{vols.text}\n')
    print('\r', '[Records saved:', hCount, '\\', bCount, ']', end='')
    hCount = hCount + 1

end = time.time()
timeTaken = round(((end - start)/60))
print(f'\nTime taken: {timeTaken} minutes')

df = pd.read_csv(fname, names=['MMSID', '300a'], sep='\t', header=None)
localFile = FileLink(fname, result_html_prefix='CSV file: ')
display(localFile)
display(df)
